In [1]:
import pandas as pd

file_path = '../../data/school_health_preprocessed.csv'

try:
    data = pd.read_csv(file_path, encoding='utf-8')
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인하세요.")


print(data[['하루30분이상운동']].isnull().sum())

cleaned_data = data.dropna(subset=['BMI', '하루30분이상운동'])
print(f"Cleaned data size: {cleaned_data.shape}")

# 성별로 데이터 분리
male_data = cleaned_data[cleaned_data['성별'] == '남']
female_data = cleaned_data[cleaned_data['성별'] == '여']


데이터 로드 성공!
하루30분이상운동    35081
dtype: int64
Cleaned data size: (49790, 111)


In [3]:
import pandas as pd

# 필요한 열 추출
df = data[['BMI', '주3회이상운동', '학교급', '성별', '하루30분이상운동']]

# '주3회이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_3times = df.dropna(subset=['BMI', '주3회이상운동', '학교급', '성별'])
print(f"Cleaned data size for 주3회이상운동: {cleaned_data_exercise_3times.shape}")
print(cleaned_data_exercise_3times.head())

# '주3회이상운동' 매핑 : 초등학생 대상
exercise_mapping_3times = {
    1.0: '예',
    2.0: '아니오'
}

cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
print("\n[주3회이상운동 매핑 결과 (초등학생 대상)]")
print(cleaned_data_exercise_3times[['BMI', '주3회이상운동', '주 3회 운동여부','성별','학교급']].head())

# '하루30분이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_30min = df.dropna(subset=['BMI', '하루30분이상운동', '학교급', '성별'])
print(f"Cleaned data size for 하루30분이상운동: {cleaned_data_exercise_30min.shape}")
print(cleaned_data_exercise_30min.head())

# '하루30분이상운동' 매핑 : 중, 고등학생 대상
exercise_mapping_30min = {
    1.0: '거의 안함',
    2.0: '주에 1~2회',
    3.0: '주에 3~4회',
    4.0: '주에 5회 이상'
}

cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)
print("\n[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]")
print(cleaned_data_exercise_30min[['BMI', '하루30분이상운동', '운동빈도','성별','학교급']].head())


Cleaned data size for 주3회이상운동: (32715, 5)
         BMI  주3회이상운동 학교급 성별  하루30분이상운동
0  15.667728      2.0   초  여        NaN
1  15.644444      2.0   초  여        NaN
2  15.159148      2.0   초  여        NaN
3  22.701628      2.0   초  여        NaN
4  14.870556      1.0   초  여        NaN

[주3회이상운동 매핑 결과 (초등학생 대상)]
         BMI  주3회이상운동 주 3회 운동여부 성별 학교급
0  15.667728      2.0       아니오  여   초
1  15.644444      2.0       아니오  여   초
2  15.159148      2.0       아니오  여   초
3  22.701628      2.0       아니오  여   초
4  14.870556      1.0         예  여   초
Cleaned data size for 하루30분이상운동: (49790, 5)
             BMI  주3회이상운동 학교급 성별  하루30분이상운동
29623  15.238947      NaN   중  남        3.0
29624  20.342891      NaN   중  여        1.0
29625  24.689836      NaN   중  남        3.0
29626  20.877623      NaN   중  여        2.0
29627  17.201060      NaN   중  여        1.0

[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]
             BMI  하루30분이상운동     운동빈도 성별 학교급
29623  15.238947        3.0  주에 3~4회  남   중
29624  20.342891        1.0  

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/1123144500.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/1123144500.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)


In [8]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

# 독립 변수 준비
cleaned_data_exercise_30min['성별_encoded'] = cleaned_data_exercise_30min['성별'].map({'남': 1, '여': 0})  # 성별을 숫자로 변환
cleaned_data_exercise_30min['학교급_encoded'] = cleaned_data_exercise_30min['학교급'].map({'중': 1, '고': 2})  # 학교급을 숫자로 변환
cleaned_data_exercise_30min['운동빈도_encoded'] = cleaned_data_exercise_30min['운동빈도'].map({
    '거의 안함': 1,
    '주에 1~2회': 2,
    '주에 3~4회': 3,
    '주에 5회 이상': 4
})  # 운동빈도를 숫자로 변환

# 종속 변수 (BMI)와 독립 변수
X = cleaned_data_exercise_30min[['운동빈도_encoded', '성별_encoded', '학교급_encoded']]
y = cleaned_data_exercise_30min['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 모델 생성 및 적합
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[회귀 분석 결과]")
print(results.summary())



[회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     684.9
Date:                Tue, 10 Dec 2024   Prob (F-statistic):               0.00
Time:                        07:41:51   Log-Likelihood:            -1.3456e+05
No. Observations:               49790   AIC:                         2.691e+05
Df Residuals:                   49786   BIC:                         2.692e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           19.3939      0.067  

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/2627945790.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['성별_encoded'] = cleaned_data_exercise_30min['성별'].map({'남': 1, '여': 0})  # 성별을 숫자로 변환
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/2627945790.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['학교급_encoded'] = cleaned_data_exercise_30min['학교급'].map({'중': 1, '고': 2})  # 학교급을 숫자로 변환
/var/folders/qz/d378mfsj2

In [9]:
import pandas as pd
import statsmodels.api as sm

# 독립 변수 준비
cleaned_data_exercise_3times['운동여부_encoded'] = cleaned_data_exercise_3times['주 3회 운동여부'].map({'예': 1, '아니오': 0})  # 주 3회 운동 여부를 숫자로 변환
cleaned_data_exercise_3times['성별_encoded'] = cleaned_data_exercise_3times['성별'].map({'남': 1, '여': 0})  # 성별을 숫자로 변환

# 종속 변수 (BMI)와 독립 변수
X = cleaned_data_exercise_3times[['운동여부_encoded', '성별_encoded']]
y = cleaned_data_exercise_3times['BMI']

# 상수항 추가
X = sm.add_constant(X)

# 회귀 모델 생성 및 적합
model = sm.OLS(y, X)
results = model.fit()

# 결과 출력
print("\n[회귀 분석 결과]")
print(results.summary())



[회귀 분석 결과]
                            OLS Regression Results                            
Dep. Variable:                    BMI   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     326.3
Date:                Tue, 10 Dec 2024   Prob (F-statistic):          4.77e-141
Time:                        07:42:36   Log-Likelihood:                -88837.
No. Observations:               32715   AIC:                         1.777e+05
Df Residuals:                   32712   BIC:                         1.777e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           18.2322      0.038  

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/3308816208.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['운동여부_encoded'] = cleaned_data_exercise_3times['주 3회 운동여부'].map({'예': 1, '아니오': 0})  # 주 3회 운동 여부를 숫자로 변환
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_11567/3308816208.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['성별_encoded'] = cleaned_data_exercise_3times['성별'].map({'남': 1, '여': 0})  # 성별을 숫자로 변환
